In [12]:
###############################################################################
# This notebook provides some cleanup tools                                   #
#                                                                             #
# WARNING!!! This notebook directly writes to the database. You need to know  #
# exactly what you are doing as you may mistakenly delete important data.     #
# Best to only work on development database.                                  #
###############################################################################

# Core stuff
import os
from pathlib import Path
import re
import json
import datetime

# Data stuff
import pandas as pd # Data analysis
import xlrd # excel 
import pyodbc # SQL DBAPI (lower level API)

# Initial setup
test = 'MISAT' # NMCT
country = 'RMI' # FSM
cwd = os.getcwd()

# Configuration
with open('config.json', 'r') as file:
     config = json.load(file)
config

# Establish a database server connection
conn = """
    Driver={{ODBC Driver 17 for SQL Server}};
    Server={},{};
    Database={};
    authentication=SqlPassword;UID={};PWD={};
    TrustServerCertificate=yes;
    autocommit=True
    """.format(config['server_ip'], config['server_port'], config['database'], config['uid'], config['pwd'])

sql_conn = pyodbc.connect(conn)

# With SQLAlchemy
from sqlalchemy import create_engine, Table, delete # SQL general
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, DateTime, Integer, Unicode
from sqlalchemy.dialects.mssql import TIMESTAMP
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('mssql+pyodbc://'+config['uid']+':'+config['pwd']+'@'+config['server_ip']+':'+config['server_port']+'/'+config['database']+'?driver=ODBC+Driver+17+for+SQL+Server') #SQL+Server+Native+Client+10.0
Base = declarative_base()
metadata = Base.metadata

In [2]:
# Load what we needs from EMIS database
query_nationalities = """
SELECT [codeCode]
      ,[codeDescription]
      ,[codeSeq]
  FROM [dbo].[lkpNationality]
"""
                          
df_nationalities = pd.read_sql(query_nationalities, sql_conn)
display(df_nationalities)

,codeCode,codeDescription,codeSeq
0,TL,Timor-Leste,999
1,TM,Turkmenistan,999
2,TN,Tunisia,999
3,TO,Tonga,999
4,TR,Turkey,999
...,...,...,...
251,TF,French Southern Territories,999
252,TG,Togo,999
253,TH,Thailand,999
254,TJ,Tajikistan,999


In [3]:
def load_excel_to_df(filename):
    """Loads an Excel filename to a Pandas DataFrame.

    Parameters
    ----------
    filename : str, required
        The filename of the excel file to load

    Raises
    ------
    NotImplementedError
        Could raise unknown error. Implement if it happens
    
    Returns
    -------
    DataFrame
    """
    file_path = Path(filename)
    file_extension = file_path.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(filename, index_col=None, header=0, engine='openpyxl')
    elif file_extension == 'xls':
        df = pd.read_excel(filename, index_col=None, header=0)
    elif file_extension == 'csv':
        df = pd.read_csv(filename, index_col=None, header=0)
    else:
        raise Exception("File not supported")

    return df

def listDuplicates(d):
    """Looks at a typical lookup dict and finds duplicate values. This can then be observed to 
    any invalid duplicates. For example, same value should be same key but with data error (misspelling, etc.).
    But a duplicate value with completely different keys (e.g. two completely different instituation with same
    value code is invalid and must be adjusted)
    """
    l = list(d.values())
    s = set([x for x in l if l.count(x) > 1])

    for k,v in d.items():
        if v in s:
            print("{}: {}".format(k, d[k]))

In [41]:
# Load a single scholarship data workbook
cwd = os.getcwd()
filename = os.path.join(cwd, 'data/FSM/Scholarship Data for FY 17.xlsx')

df_scholarship = load_excel_to_df(filename)
print('df_scholarship')
display(df_scholarship)

df_scholarship


,Last Name,First Name,State,G.L.,GPA,Institution,Major,Cost of Attendance,Amount Awarded,Semester,MS #,Note,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,Lawechresh,Tracy,Yap,Sophomore,2.650,University of Hawaii-Manoa,Public Health,39049.0,4000.0,Fall 2016,NS17-01,NaN,NaN,NaN,NaN,NaN,NaN
1,Sigrah,Keeshea,Kosrae,Junior,2.550,Chaminade University,Environmental Studies,39777.0,4000.0,Fall 2016,NS17-02,NaN,NaN,NaN,NaN,NaN,NaN
2,Lekka,Noriekka,Pohnpei,Senior,2.188,University of Guam,Health Science,14171.0,4000.0,Fall 2016,NS17-03,NaN,NaN,NaN,NaN,NaN,NaN
3,Roby,Rofino,Pohnpei,Junior,2.540,University of Hawaii-Hilo,Psychology,15844.0,4000.0,Spring 2017,NS17-04,NaN,NaN,NaN,NaN,NaN,NaN
4,Tharngan,Eric,Yap,Junior,2.940,Guam Community College,Computer Networking,10614.0,4000.0,Fall 2016,NS17-05,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
###############################################################################
# Cleanup data
###############################################################################

# Drop columns and rows where all of it is NaN
df_scholarship = df_scholarship.dropna(axis=1, how='all')
df_scholarship = df_scholarship.dropna(axis=0, how='all')

# Add the year to the DF
df_scholarship['Year'] = 2017

# Cleanup states
states_lookup_byname = {
    'YAP': 'YAP',
    'KSA': 'KSA',
    'PNI': 'CHK',
    'CHK': 'CHK',
    'Yap': 'YAP',
    'Kosrae': 'KSA', 
    'Pohnpei': 'PNI', 
    'Chuuk': 'CHK', 
    'Yap/USA': 'YAP', 
    'Yap/Marshall': 'YAP',
    'USA/Yap': 'YAP', 
    'Pohnpei ': 'PNI'
}

df_scholarship['State'] = df_scholarship['State'].map(states_lookup_byname)

# Cleanup grade levels
gradelevels_hs_lookup_byname = {
    'Sophomore': 'Year 2', 
    'Junior': 'Year 3', 
    'Senior': 'Year 4', 
    'Freshman': 'Year 1', 
    'Postgraduate': 'Postgraduate',
    'Graduate': 'Graduate' 
}
gradelevels_college_lookup_byname = {
    'Sophomore': 'G10', 
    'Junior': 'G11', 
    'Senior': 'G12', 
    'Freshman': 'G9', 
    'Postgraduate': 'Postgraduate',
    'Graduate': 'Graduate'
}

df_scholarship['ClassLevelHS'] = df_scholarship['G.L.'].map(gradelevels_hs_lookup_byname)
df_scholarship['ClassLevelCollege'] = df_scholarship['G.L.'].map(gradelevels_college_lookup_byname)

# Clean GPA
# Check GPA within range
gpa = df_scholarship['GPA'].between(0,4).any()
if not gpa:
    print('GPA is not within range: ')
# For now put GPA in [schoGPA] since we may not have data tracked by actual semester to put in posGPA
df_scholarship['schoGPA'] = df_scholarship['GPA']
    
# Cleanup institution
institution_lookup_byname = {
    'University of Hawaii-Manoa': 'UOH',
    'Chaminade University': 'CHU',
    'University of Guam': 'UOG',
    'University of Hawaii-Hilo': 'UOH',
    'Guam Community College': 'GCC',
    'University of Hawaii Manoa': 'UOH',
    'College of Micronesia-FSM': 'COM',
    'Pacific Islands University': 'PIU',
    'Eastern Oregon University': 'EOU',
    'Hawaii Community College': 'HCC',
    'Cardinal Stritch University': 'CSU',
    'Grand Rapids Community College': 'GRCC',
    'Park University': 'PU',
    'University of Hawaii Hilo': 'UOH',
    "Saint Martin's University": 'SMU', 
    'Creighton University': 'CRU',
    'Wheeling Jesuit University': 'WJU',
    'University of Scranton': 'US',
    'University of Oregon': 'UO',
    'Western Kentucky University': 'UKU',
    'Hawaii Pacific University': 'HPU',
    'University of Hawaii Hilo/Hawaii Community College': 'UOH',
    'University of Reno': 'UON',
    'Canisius College': 'CC',
    'Southern Utah University': 'SUU',
    'Mount Mary University': 'MMU',
    'The University of the South Pacific': 'USP',
    'Allegheny College': 'AC',
    'Southern New Hampshire University': 'SNHU',
    'University of Tennessee Knoxville': 'UTK',
    'Fiji National University': 'FNU',
    'Kapiolani Community College': 'KCC',
    'Rocky Mountain College': 'RMC',
    'University of Texas, San Antonio': 'UOT',
    'Walla Walla Community College': 'WWCC',
    'Harvest Baptist Bible College': 'HBBC',
    'Southern Adventist University': 'SAU',
    'Pacific Union College': 'PUC',
    'Miramar College': 'MC',
    'Walla Walla University': 'WWU',
    'Bob Jones University': 'BJU',
    'Univerity of Guam': 'UOG',
    'University of Montana': 'UOM', 
    "Saint Joseph's University": 'STJU',
    'University of Hawii Hilo': 'UOH', 
    "Sain Martin's University": 'SMU',
    'University of Nevada, Reno': 'UON',
    'Bossier Parish Community College': 'BPCC',
    "St. Martin's University": 'SMU', 
    'Southwestern University': 'SU',
    'Pennsylvania State University': 'PSU', 
    "St. Peter's University": 'STPU',
    'International Baptist College & Seminary': 'IBCS',
    'Western Oklahoma State College': 'WOSC',
    'University of Texas at San Antonio': 'UOT',
    'Sanata Dharma University': 'SDU',
    'Ritsumeikan Asian Pacific University': 'RAPU',
    'University of South Pacific': 'USP',
    'University of Hawaii Maui College': 'UOH',
    'Andrews University': 'AU',
    'Seattle Pacific University': 'SPU',
    'Le Moyne College': 'LMC'
}
# For verification of unique codes
#listDuplicates(gradelevels_college_lookup_byname)
df_scholarship['instCode'] = df_scholarship['Institution'].map(institution_lookup_byname)

# Draft Lookup by code (draft because it produces it from above dict with its alternative spellings, etc.)
# Still need to make an official version from this for loading into the DB
institution_lookup_draft = {v:k for k,v in institution_lookup_byname.items()}

institution_lookup = {
    'UOH': 'University of Hawaii',
    'CHU': 'Chaminade University',
    'UOG': 'Univerity of Guam',
    'GCC': 'Guam Community College',
    'COM': 'College of Micronesia',
    'PIU': 'Pacific Islands University',
    'EOU': 'Eastern Oregon University',
    'HCC': 'Hawaii Community College',
    'CSU': 'Cardinal Stritch University',
    'GRCC': 'Grand Rapids Community College',
    'PU': 'Park University',
    'SMU': "St. Martin's University",
    'CRU': 'Creighton University',
    'WJU': 'Wheeling Jesuit University',
    'US': 'University of Scranton',
    'UO': 'University of Oregon',
    'UKU': 'Western Kentucky University',
    'HPU': 'Hawaii Pacific University',
    'CC': 'Canisius College',
    'SUU': 'Southern Utah University',
    'MMU': 'Mount Mary University',
    'USP': 'University of South Pacific',
    'AC': 'Allegheny College',
    'SNHU': 'Southern New Hampshire University',
    'UTK': 'University of Tennessee Knoxville',
    'FNU': 'Fiji National University',
    'KCC': 'Kapiolani Community College',
    'RMC': 'Rocky Mountain College',
    'UOT': 'University of Texas at San Antonio',
    'WWCC': 'Walla Walla Community College',
    'HBBC': 'Harvest Baptist Bible College',
    'SAU': 'Southern Adventist University',
    'PUC': 'Pacific Union College',
    'MC': 'Miramar College',
    'WWU': 'Walla Walla University',
    'BJU': 'Bob Jones University',
    'UOM': 'University of Montana',
    'STJU': "Saint Joseph's University",
    'UON': 'University of Nevada, Reno',
    'BPCC': 'Bossier Parish Community College',
    'SU': 'Southwestern University',
    'PSU': 'Pennsylvania State University',
    'STPU': "St. Peter's University",
    'IBCS': 'International Baptist College & Seminary',
    'WOSC': 'Western Oklahoma State College',
    'SDU': 'Sanata Dharma University',
    'RAPU': 'Ritsumeikan Asian Pacific University',
    'AU': 'Andrews University',
    'SPU': 'Seattle Pacific University',
    'LMC': 'Le Moyne College'
}

# Clean semesters
# TODO Remains open question about handling of semesters to their EMIS value
semesters_lookup = {
    'Fall 2016': 1, #??? 
    'Spring 2017': 2,
    'Summer 2017': 3,
    'Fall 2017': 1 #???
}

df_scholarship['posSemester'] = df_scholarship['Semester'].map(semesters_lookup)

# Simply put Note in schoNote for now
df_scholarship.rename(columns={"Note": "schoNote"}, inplace=True)

# Amount awarded, is this the amount awarded for the semester? or the whole scholarship?
df_scholarship.rename(columns={"Amount Awarded": "posAmount"}, inplace=True)

# MS# ?

display(df_scholarship)

,Last Name,First Name,State,G.L.,GPA,Institution,Major,Cost of Attendance,posAmount,Semester,MS #,schoNote,Year,ClassLevelHS,ClassLevelCollege,schoGPA,instCode,posSemester
0,Lawechresh,Tracy,YAP,Sophomore,2.650,University of Hawaii-Manoa,Public Health,39049.0,4000.0,Fall 2016,NS17-01,NaN,2017,Year 2,G10,2.650,UOH,1
1,Sigrah,Keeshea,KSA,Junior,2.550,Chaminade University,Environmental Studies,39777.0,4000.0,Fall 2016,NS17-02,NaN,2017,Year 3,G11,2.550,CHU,1
2,Lekka,Noriekka,CHK,Senior,2.188,University of Guam,Health Science,14171.0,4000.0,Fall 2016,NS17-03,NaN,2017,Year 4,G12,2.188,UOG,1
3,Roby,Rofino,CHK,Junior,2.540,University of Hawaii-Hilo,Psychology,15844.0,4000.0,Spring 2017,NS17-04,NaN,2017,Year 3,G11,2.540,UOH,2
4,Tharngan,Eric,YAP,Junior,2.940,Guam Community College,Computer Networking,10614.0,4000.0,Fall 2016,NS17-05,NaN,2017,Year 3,G11,2.940,GCC,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,Noda,Gracelynn,KSA,Sophomore,3.493,University of Guam,Business Administration,14171.0,4000.0,Fall 2017,NS17-590,NaN,2017,Year 2,G10,3.493,UOG,1
589,Simina,Leyann,CHK,Senior,3.113,University of Guam,Public Health/ Nutrition,10738.5,5000.0,Fall 2017,NS17-591,NaN,2017,Year 4,G12,3.113,UOG,1
590,Tammed,Marietta,YAP,Senior,2.759,University of Guam,Accounting,10959.5,4000.0,Fall 2017,NS17-592,NaN,2017,Year 4,G12,2.759,UOG,1
591,Ligohr,Denzel,CHK,Sophomore,3.840,Hawaii Pacific University,International Studies,47536.0,3500.0,Fall 2017,NS17-593,NaN,2017,Year 2,G10,3.840,HPU,1


In [31]:
df_scholarship['MS #'].unique()

array(['NS17-01', 'NS17-02', 'NS17-03', 'NS17-04', 'NS17-05', 'NS17-06',
       'NS17-07', 'NS17-08', 'NS17-09', 'NS17-10', 'NS17-11', 'NS17-12',
       'NS17-13', 'NS17-14', 'NS17-15', 'NS17-16', 'NS17-17', 'NS17-18',
       'NS17-19', 'NS17-20', 'NS17-21', 'NS17-22', 'NS17-23', 'NS17-24',
       'NS17-25', 'NS17-26', 'NS17-27', 'NS17-28', 'NS17-29', 'NS17-30',
       'NS17-31', 'NS17-32', 'NS17-33', 'NS17-34', 'NS17-35', 'NS17-36',
       'NS17-37', 'NS17-38', 'NS17-39', 'NS17-40', 'NS17-41', 'NS17-42',
       'NS17-43', 'NS17-44', 'NS17-45', 'NS17-46', 'NS17-47', 'NS17-48',
       'NS17-49', 'NS17-50', 'NS17-51', 'NS17-52', 'NS17-53', 'NS17-54',
       'NS17-55', 'NS17-56', 'NS17-57', 'NS17-58', 'NS17-59', 'NS17-60',
       'NS17-61', 'NS17-62', 'NS17-63', 'NS17-64', 'NS17-65', 'NS17-66',
       'NS17-67', 'NS17-68', 'NS17-69', 'NS17-70', 'NS17-71', 'NS17-72',
       'NS17-73', 'NS17-74', 'NS17-75', 'NS17-76', 'NS17-77', 'NS17-78',
       'NS17-79', 'NS17-80', 'NS17-81', 'NS17-82', 

In [25]:
###############################################################################
# ScholarshipInsitution table                                                 #
###############################################################################

# ORM model 
#class ScholarshipInstitution(Base):
#    __tablename__ = 'ScholarshipInstitutions'

#    instCode = Column(Unicode(10), primary_key=True)
#    instName = Column(Unicode(100), nullable=False)
#    instCountry = Column(Unicode(2), nullable=False)
#    instTermsPerYear = Column(Integer)
#    pCreateUser = Column(Unicode(50))
#    pCreateDateTime = Column(DateTime)
#    pEditUser = Column(Unicode(50))
#    pEditDateTime = Column(DateTime)
#    pRowversion = Column(TIMESTAMP)
    
# Get unique list of institute names to gather their countries (manually)
insts = list(institution_lookup.values())
insts.sort()
insts

# Here they are, now get the countries for each
institution_countries_lookup = {
    'Allegheny College': 'US',
    'Andrews University': 'US',
    'Bob Jones University': 'US',
    'Bossier Parish Community College': 'US',
    'Canisius College': 'US',
    'Cardinal Stritch University': 'US',
    'Chaminade University': 'US',
    'College of Micronesia': 'FM',
    'Creighton University': 'US',
    'Eastern Oregon University': 'US',
    'Fiji National University': 'FJ',
    'Grand Rapids Community College': 'US',
    'Guam Community College': 'US',
    'Harvest Baptist Bible College': 'US',
    'Hawaii Community College': 'US',
    'Hawaii Pacific University': 'US',
    'International Baptist College & Seminary': 'PH',
    'Kapiolani Community College': 'US',
    'Le Moyne College': 'US',
    'Miramar College': 'US',
    'Mount Mary University': 'US',
    'Pacific Islands University': 'US',
    'Pacific Union College': 'US',
    'Park University': 'US',
    'Pennsylvania State University': 'US',
    'Ritsumeikan Asian Pacific University': 'JP',
    'Rocky Mountain College': 'US',
    "Saint Joseph's University": 'US',
    'Sanata Dharma University': 'ID',
    'Seattle Pacific University': 'US',
    'Southern Adventist University': 'US',
    'Southern New Hampshire University': 'US',
    'Southern Utah University': 'US',
    'Southwestern University': 'US',
    "St. Martin's University": 'US',
    "St. Peter's University": 'US',
    'Univerity of Guam': 'US',
    'University of Hawaii': 'US',
    'University of Montana': 'US',
    'University of Nevada, Reno': 'US',
    'University of Oregon': 'US',
    'University of Scranton': 'US',
    'University of South Pacific': 'FJ',
    'University of Tennessee Knoxville': 'US',
    'University of Texas at San Antonio': 'US',
    'Walla Walla Community College': 'US',
    'Walla Walla University': 'US',
    'Western Kentucky University': 'US',
    'Western Oklahoma State College': 'US',
    'Wheeling Jesuit University': 'US'
}

institutions = {
    'instCode': institution_lookup.keys(),
    'instName': institution_lookup.values()
}

df_scholarship_institutions = pd.DataFrame(data=institutions)

# Add countries
df_scholarship_institutions['instCountry'] = df_scholarship_institutions['instName'].map(institution_countries_lookup)

# All institutes to default to 3 terms per year
df_scholarship_institutions['instTermsPerYear'] = 3

# Add changed tracking data
df_scholarship_institutions['pCreateUser'] = config['user']
df_scholarship_institutions['pCreateDateTime'] = datetime.datetime.utcnow()
df_scholarship_institutions['pEditUser'] = config['user']
df_scholarship_institutions['pEditDateTime'] = datetime.datetime.utcnow()

df_scholarship_institutions.sort_values(by=['instName'], inplace=True)
display(df_scholarship_institutions.head(3)) #.head(3)

# Write data directly to EMIS database
# Delete all existing data from table first
stmt = (
    delete(ScholarshipInstitution)
)
result = engine.execute(stmt)

# Re-insert from DataFrame
df_scholarship_institutions.to_sql('ScholarshipInstitutions', engine, if_exists='append', index=False)

,instCode,instName,instCountry,instTermsPerYear,pCreateUser,pCreateDateTime,pEditUser,pEditDateTime
22,AC,Allegheny College,US,3,ghachey@nuzusys,2021-05-25 07:14:17.135082,ghachey@nuzusys,2021-05-25 07:14:17.136344
47,AU,Andrews University,US,3,ghachey@nuzusys,2021-05-25 07:14:17.135082,ghachey@nuzusys,2021-05-25 07:14:17.136344
35,BJU,Bob Jones University,US,3,ghachey@nuzusys,2021-05-25 07:14:17.135082,ghachey@nuzusys,2021-05-25 07:14:17.136344
